# Setup

In [ ]:
%%time
import tensorflow as tf
from tensorflow.keras import layers, models, losses, Model, optimizers

In [3]:
# import os for stuff
import os
import re
import string
import numpy as np
import pandas as pd
import pickle
from pprint import pprint
import matplotlib.pyplot as plt

from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection  import KFold
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

# to supress invalid warning 
old_err_state = np.seterr(invalid ='ignore')

In [ ]:
# nltk
import nltk
nltk.download(['stopwords','punkt'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

porterStemmer = nltk.stem.PorterStemmer()

In [6]:
# seop constant 
sep = '\\' if os.name == 'nt' else '/'

In [7]:
# simple function
def get_folder_list(path) :
    # get list of dir from path
    dir_list = None
    for (root,dirs,files) in os.walk(path, topdown=True):
        dir_list = dirs
        break
    return dir_list

def get_file_list(path) :
    # get list of file in path
    file_list = None
    for (root,dirs,files) in os.walk(path, topdown=True):
        file_list = files
        break
    return file_list



## Dataset Preparation

In [8]:
def text_dataset_from_directory_to_dataframe(path) :
    # return df
    # text_dataset_from_directory function
    
    #dset_path = os.getcwd() + sep +"data"+sep+"bbc"
    dset_path = path
    dset_name = get_folder_list(dset_path)

    # key  : class name    -> str
    # value: list of files -> list of str
    file_dict = dict()


    # get list of dir from path
    for class_name in dset_name :
        class_path = dset_path + sep + class_name
        file_dict[class_name] = get_file_list(class_path)
        # print(get_file_list(class_path))

    # print(file_dict)

    # key   : class name -> str
    # value : list of text -> list of str
    text_dict = dict()


    for class_name, f_name_list in file_dict.items():
        # print(key, values)
        # print(class_name)
        # print(f_name_list)
        # raw string
        text_list = []
        for txt_name in f_name_list :
            # print(txt_name)
            # print(class_name)
            f_path = dset_path + sep + class_name + sep + txt_name
            with open(f_path ,'r') as f :
                ## \n\n will be .
                tt = re.sub('\n\n', '.', f.read())
                text_list.append(tt)

        text_dict[class_name] = text_list

    # print(text_dict)

    # list of tuple -> (text, label)
    zipped_list = []
    for class_name, text_list in text_dict.items() :
        temp = list(zip(text_list, [class_name]*len(text_list)))
        # print(temp[:3])
        zipped_list = zipped_list + temp


    df = pd.DataFrame(zipped_list, columns=['text', 'label'])
    
    return df
    # t = dset_path + sep + dset_name[0] + sep + '001.txt'
    # with open(t,'r') as f :
    #     txt = f.read()

    # print(txt)

In [9]:
dataset_path = os.getcwd() + sep +"data"+sep+"bbc"

In [10]:
df = text_dataset_from_directory_to_dataframe(dataset_path)
df

,text,label
0,Ad sales boost Time Warner profit.Quarterly pr...,business
1,Dollar gains on Greenspan speech.The dollar ha...,business
2,Yukos unit buyer faces loan claim.The owners o...,business
3,High fuel prices hit BA's profits.British Airw...,business
4,Pernod takeover talk lifts Domecq.Shares in UK...,business
...,...,...
2220,BT program to beat dialler scams.BT is introdu...,tech
2221,Spam e-mails tempt net shoppers.Computer users...,tech
2222,Be careful how you code.A new European directi...,tech
2223,US cyber security chief resigns.The man making...,tech


In [11]:
# shuffle the DataFrame rows
shuff = df.sample(frac = 1, random_state=4473, ignore_index=True)

# label encoding
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
shuff['label_cat'] = labelencoder.fit_transform(shuff['label'])

In [12]:
# for sanity
class_name_list = list(labelencoder.classes_)
class_dict = {}

for class_name in class_name_list :
    lbl_num = labelencoder.transform([class_name])[0]
    class_dict[lbl_num] = class_name
    
print(class_dict)

{0: 'business', 1: 'entertainment', 2: 'politics', 3: 'sport', 4: 'tech'}


# Preprocessing

In [13]:
%%time
def custom_standardization(input_data, verbose= False):
    # print original string
    if verbose :
        print("original string: " ,input_data)
    
    # lowering
    lowercase = input_data.lower()
    
    # print lowercase string
    if verbose :
        print('lower string: ', lowercase)
    
    
    # clean the text
    stripped = re.sub(" #39;", "\'", lowercase)
    stripped = re.sub(" quot;", "\"", stripped)
    # remove stopword
    # regex magic : r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*'
    stripped = re.sub(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*',"", stripped)
    
    # print remove stopword string
    if verbose :
        print('Stopword removal: ', stripped)
    
    # strip punctuiation
    stripped = re.sub(f"[{re.escape(string.punctuation)}]", " ", stripped)
    # remove double space
    # two or more whitespace
    stripped = re.sub(r"(\s\s*)", " ", stripped)
    stripped = re.sub(r"(\s+$)", "", stripped)
    
    # tokenize
    tokens = word_tokenize(stripped)

    # print tokenize
    if verbose :
        print("toknize: ", tokens)
    
    # stemming
    temp_cont = [porterStemmer.stem(word) for word in tokens]
    
    # print stemmed
    if verbose:
        print("stemmed: ", temp_cont)
    
    porter_    = ' '.join(temp_cont)
    return porter_

CPU times: total: 0 ns
Wall time: 0 ns


In [14]:
%%time
c11_start = datetime.now()
# clean text data
clean_text = list(map(custom_standardization, shuff['text'].tolist()))
shuff['clean_text'] = clean_text

c11_duration = datetime.now() - c11_start
print("cell:map_cleaning-text \nExecution Time : {0:.2f} s".format(c11_duration.total_seconds()))

cell:map_cleaning-text 
Execution Time : 18.35 s
CPU times: total: 18.1 s
Wall time: 18.3 s


In [15]:
shuff

,text,label,label_cat,clean_text
0,Looks and music to drive mobiles.Mobile phones...,tech,4,look music drive mobil mobil phone still enjoy...
1,BT offers equal access to rivals.BT has moved ...,business,0,bt offer equal access rival bt move pre empt p...
2,High fuel prices hit BA's profits.British Airw...,business,0,high fuel price hit ba profit british airway b...
3,UK firms 'embracing e-commerce'.UK firms are e...,politics,2,uk firm embrac e commerc uk firm embrac intern...
4,News Corp makes $5.4bn Fox offer.News Corporat...,business,0,news corp make 5 4bn fox offer news corpor see...
...,...,...,...,...
2220,Microsoft seeking spyware trojan.Microsoft is ...,tech,4,microsoft seek spywar trojan microsoft investi...
2221,Labour battle plan 'hides Blair'.The Tories ha...,politics,2,labour battl plan hide blair tori accus toni b...
2222,Singer Sizzla jailed for swearing.Reggae star ...,entertainment,1,singer sizzla jail swear regga star sizzla who...
2223,Blog reading explodes in America.Americans are...,tech,4,blog read explod america american becom avid b...


## Build TF-IDF SKlearn

In [16]:
%%time
# CELLNUM  : 13
# CELLNAME : lucky-charm

c13_start = datetime.now()
# generate & fit
vectorizer = TfidfVectorizer(min_df=0.01, 
                             max_df = 0.8, 
                             ngram_range=(1, 1))
vectorizer.fit(shuff['clean_text'])
dim_size = vectorizer.get_feature_names_out().shape[0]
print("Vector Size : ", dim_size)
c13_duration = datetime.now() - c13_start
print("cell:train_tf-idf \nExecution Time : {0:.2f} s".format(c13_duration.total_seconds()))

Vector Size :  2510
cell:train_tf-idf 
Execution Time : 0.42 s
CPU times: total: 406 ms
Wall time: 417 ms


## K-Fold

In [17]:
K5_fold = KFold(n_splits=5)
fold = []
# fold -> [[train_idex, val_index, test_index], ...,[train_idex, val_index, test_index]]

for train_index, test_index in K5_fold.split(shuff['clean_text']):
    val_index = test_index[int(len(test_index) / 2): ]
    test_index = test_index[:int(len(test_index)/2)]
#     print("TRAIN:", train_index, "VAL:", val_index, "TEST:", test_index)
    fold.append([train_index, val_index, test_index])

# ANN

## Build NN
Build a NN from predetermined composition <br>
Need `input_size` as a parameter          <br>


In [18]:
%time
# CELLNUM  : 21
# CELLNAME : nana
# Desc     : nn get it ?

def build_nn(input_size:int):
    input_layer = layers.Input(shape=(input_size,))

    hdn  = layers.Dense(900, activation='selu') (input_layer)
    hdn = layers.BatchNormalization(momentum=0.99, epsilon=0.0001)(hdn)
    hdn  = layers.Dense(600, activation='elu') (hdn)
    hdn = layers.BatchNormalization(momentum=0.99, epsilon=0.0001)(hdn)
    hdn  = layers.Dense(900, activation='selu') (hdn)
    hdn  = layers.Dense(700, activation='selu') (hdn)

    # using logits so this is how the output look
    predictions = layers.Dense(20, name="predictions")(hdn)
    
    model = tf.keras.Model(input_layer, predictions)
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # Compile the model with binary crossentropy loss and an adam optimizer.
    model.compile(loss= loss,
              optimizer="adam", 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    
    return model

CPU times: total: 0 ns
Wall time: 0 ns


## Metrics for ANN

In [19]:
# function to classification_report homebrew
def _report_classification(confusion_matrix, class_name= class_dict, as_df=True) :
    # class_name -> a dict with int as key and str as value
    
    kf_tbl = {}
    for i in range(confusion_matrix.shape[0]) :
        c_name = class_name[i]
        kf_tbl[c_name] = {}
        # TP 
        kf_tbl[c_name]["TP"] = confusion_matrix[i,i]
        # FN
        # baris koma belakang
        kf_tbl[c_name]["FN"] = np.sum(confusion_matrix[i, :]) - confusion_matrix[i,i]
        # FP
        # klo liat kolom koma depan
        kf_tbl[c_name]["FP"] = np.sum(confusion_matrix[: ,i]) - confusion_matrix[i,i]
        # FN
        kf_tbl[c_name]["TN"] = np.sum(confusion_matrix) - (kf_tbl[c_name]["TP"] + kf_tbl[c_name]["FN"] + kf_tbl[c_name]["FP"])
        # Precision
        kf_tbl[c_name]["Precision"] = np.nan_to_num(kf_tbl[c_name]["TP"]/(kf_tbl[c_name]["TP"] + kf_tbl[c_name]["FP"]))

        # Recall
        kf_tbl[c_name]["Recall"] = np.nan_to_num((kf_tbl[c_name]["TP"]/(kf_tbl[c_name]["TP"] + kf_tbl[c_name]["FN"])))
        # f1-score per-class
        _temp = (kf_tbl[c_name]["Precision"]* kf_tbl[c_name]["Recall"])
        _temp = _temp/(kf_tbl[c_name]["Precision"] + kf_tbl[c_name]["Recall"])
        _temp = np.nan_to_num(_temp)
        kf_tbl[c_name]["f1-score"] = np.nan_to_num(2*_temp)
    if as_df : 
        t_tbl = {"labels" : [],
                 "TP"     : [],
                 "FN"     : [],
                 "FP"     : [],
                 "TN"     : [],
                 "Precision" : [],
                 "Recall"    : [],
                 "f1-score"  : []
                }
        for key in list(kf_tbl.keys()) :
            t_tbl['labels'].append(key)
            t_tbl['TP'].append(kf_tbl[key]['TP'])
            t_tbl['FN'].append(kf_tbl[key]['FN'])
            t_tbl['FP'].append(kf_tbl[key]['FP'])
            t_tbl['TN'].append(kf_tbl[key]['TN'])
            t_tbl['Precision'].append(kf_tbl[key]['Precision'])
            t_tbl['Recall'].append(kf_tbl[key]['Recall'])
            t_tbl['f1-score'].append(kf_tbl[key]['f1-score'])
        df = pd.DataFrame.from_dict(t_tbl)
        df = df.set_index('labels')
        
        return df
    
    return kf_tbl

In [20]:
# function to convert data from _report_classification to macro-f1 and micro-f1 and weighted-f1
# support = tp + fn
def get_f1(cls_report) :
    cls_report_type = type(cls_report)
    if cls_report_type is dict :
        cls_num = len(list(cls_report.keys()))
        cls_keys = list(cls_report.keys())
        
        # macro
        # get all f1-score then just average it to cls_num
        _f1_list = []
        for key in cls_keys :
            _f1_list.append(cls_report[key]['f1-score'])
        _macro_f1 = sum(_f1_list) / cls_num
        _f1_list = None
        
        # weigthed
        # support_proportion = support / sum(support)
        # sum(f1*support_proportion)
        _f1_list = []
        _sum_support = 0
        for key in cls_keys :
            _support = cls_report[key]['TP'] + cls_report[key]['FN']
            _f1_list.append(cls_report[key]['f1-score'] * _support)
            _sum_support += _support
        _weighted_f1 = sum(_f1_list) / _sum_support
        _f1_list = None
                        
        # micro
        # all_class_TP FP  amnd FN
        # using that just count f1 normally
        _sum_TP = 0
        _sum_FP = 0
        _sum_FN = 0
        for key in cls_keys :
            _sum_TP += cls_report[key]['TP']
            _sum_FP += cls_report[key]['FP']
            _sum_FN += cls_report[key]['FN']
        _micro_f1 = _sum_TP / (_sum_TP + (0.5*(_sum_FP + _sum_FN)))
                        
        rslt = {'f1' : {
                'micro'    : _micro_f1,
                'macro'    : _macro_f1,
                'weighted' : _weighted_f1}
              }
        return rslt
    elif cls_report_type is pd.core.frame.DataFrame :
        cls_num = cls_report.shape[0]
        
        # macro
        _macro_f1 = sum(cls_report['f1-score'].tolist()) / len(cls_report['f1-score'].tolist())
        
        # weighted
        # support = tp + fn
        _cls_TP = cls_report['TP'].tolist()
        _cls_FN = cls_report['FN'].tolist()
        _cls_f1 = cls_report['f1-score'].tolist()
        
        _support = [_cls_TP[i] + _cls_FN[i] for i in range(len(_cls_TP))]
        _sum_support = sum(_support)
        
        # sum(f1[i] * (support[i] / sum(support)))
        _weighted_f1 = sum([_cls_f1[i] * (_support[i] / _sum_support) for i in range(len(_cls_f1))])
        
        # micro
        _sum_TP = sum(cls_report['TP'].tolist())
        _sum_FP = sum(cls_report['FP'].tolist())
        _sum_FN = sum(cls_report['FN'].tolist())
        _micro_f1 = _sum_TP / (_sum_TP + (0.5*(_sum_FP + _sum_FN)))
        
        rslt = {'f1' : {
                'micro'    : _micro_f1,
                'macro'    : _macro_f1,
                'weighted' : _weighted_f1}
              }
        return rslt
    else :
        err = {'f1' : {
                'micro'    : -4473,
                'macro'    : -4473,
                'weighted' : -4473}
              }
        
        return err
    

In [21]:
def _average_dataframe(list_dframe: list) :
    for i in range(len(list_dframe)) :
        if i == 0 :
            f_avg = list_dframe[i]
        else :
            f_avg = f_avg.add(list_dframe[i])

    # divide by amount of fold
    f_avg = f_avg.div(len(list_dframe))

    # round down
    f_avg['TP'] = f_avg['TP'].astype('int')
    f_avg['FN'] = f_avg['FN'].astype('int')
    f_avg['FP'] = f_avg['FP'].astype('int')
    f_avg['TN'] = f_avg['TN'].astype('int')
    
    # return an average of all dataframe
    return f_avg

# Evaluation

Evaluate the f1-measure and acc

## Function to Evaluate Vanilla Performance

In [22]:
def _eval_control(save_file=False) :

    i = 1
    t_res = []
    t_loss = []
    t_acc =[]
    
    list_dataframe = []
    for train_index, val_index, test_index in fold :
        f_cstart = datetime.now()
        # clear session _del model from memory
        tf.keras.backend.clear_session()
    
        X_train = vectorizer.transform(shuff['clean_text'][train_index]).toarray()
        X_val = vectorizer.transform(shuff['clean_text'][val_index]).toarray()
        X_test = vectorizer.transform(shuff['clean_text'][test_index]).toarray()
        
        
        y_train = shuff['label_cat'][train_index]
        y_val = shuff['label_cat'][val_index]
        y_test = shuff['label_cat'][test_index]
        
        nn_name = "single_control_{}".format(i)
        nn_model = build_nn(input_size=dim_size)
        nn_model._name = nn_name
        
        # train - fit
        fit_history = nn_model.fit(X_train, 
                                    y_train, 
                                    epochs=100, 
                                    batch_size=75, 
                                    validation_data = (X_val, y_val),
                                    verbose=0)
        # eval
        # print("Evaluate on test data ", i)
        results = nn_model.evaluate(X_test, y_test, batch_size=128, verbose=0)
        # print(nn_model.metrics_names)
        print("test loss, test acc:", results)
        pred = nn_model.predict(X_test, verbose=0)
        pred = np.argmax(pred, axis=1)
        result_df = _report_classification(confusion_matrix(y_test, pred))
        result_f1 = get_f1(result_df)
        print("fold {} f1-macro : {}".format(i, result_f1['f1']['macro']))
        t_res.append(result_f1['f1']['macro'])
        t_loss.append(results[0])
        t_acc.append(results[1])
        
        list_dataframe.append(result_df)
                
        # delete
        X_train, X_val,X_test = None, None, None
        y_train, y_val,y_test = None, None, None
        
        f_cdur = datetime.now() - f_cstart
        print("func:eval_model on fold {} \nExecution Time : {:.2f} s".format(i , f_cdur.total_seconds()))
        print()
                
        i += 1
        
    result_avg = _average_dataframe(list_dataframe)
    
    d_result = {'result' : result_avg,
                'f1-macro' : sum(t_res) / len(t_res),
                'loss' : sum(t_loss) / len(t_loss),
                'acc'  : sum(t_acc) / len(t_acc)}
    
    print('control result')
    print(d_result)
    return d_result

### Control Experiment Result

In [23]:
control_measurement = _eval_control(save_file=False)

print("AVERAGE PRECISION : ", control_measurement['result']['Precision'].mean())
print("AVERAGE RECALL : ", control_measurement['result']['Recall'].mean())

test loss, test acc: [3.372105598449707, 0.9324324131011963]
fold 1 f1-macro : 0.9306344377535535
func:eval_model on fold 1 
Execution Time : 30.25 s

test loss, test acc: [1.6693100929260254, 0.9504504799842834]
fold 2 f1-macro : 0.9442879112778323
func:eval_model on fold 2 
Execution Time : 28.24 s

test loss, test acc: [0.587522566318512, 0.9819819927215576]
fold 3 f1-macro : 0.9819615954647645
func:eval_model on fold 3 
Execution Time : 27.78 s

test loss, test acc: [1.0010404586791992, 0.9819819927215576]
fold 4 f1-macro : 0.9820274841289685
func:eval_model on fold 4 
Execution Time : 27.90 s

test loss, test acc: [0.5009213089942932, 0.977477490901947]
fold 5 f1-macro : 0.9772405051801714
func:eval_model on fold 5 
Execution Time : 28.43 s

control result
{'result':                TP  FN  FP   TN  Precision    Recall  f1-score
labels                                                       
business       47   2   1  171   0.966711  0.950302  0.957909
entertainment  35   1   1  183 

## Function to Compare with other alg

In [24]:
def _eval_svm(save_file=False) :
    i = 1
    t_res = []
    list_dataframe = []
    for train_index, val_index, test_index in fold :
        f_svmstart = datetime.now()
        
        
        X_train = vectorizer.transform(shuff['clean_text'][train_index]).toarray()
        X_val = vectorizer.transform(shuff['clean_text'][val_index]).toarray()
        X_test = vectorizer.transform(shuff['clean_text'][test_index]).toarray()


        y_train = shuff['label_cat'][train_index].tolist()
        y_val = shuff['label_cat'][val_index].tolist()
        y_test = shuff['label_cat'][test_index].tolist()
        
        svc = svm.SVC(kernel='linear',decision_function_shape='ovr')
        svc.fit(X_train, y_train)
    
        pred = svc.predict(X_test)
        
        result_df = _report_classification(confusion_matrix(y_test, pred))
        
        list_dataframe.append(result_df)
        # delete
        X_train, X_val,X_test = None, None, None
        y_train, y_val,y_test = None, None, None
        
        print('finish fold-{}'.format(i))
        i += 1
    # average all fold
    result_avg = _average_dataframe(list_dataframe)
    result_f1 = get_f1(result_avg)

    f_svmduration = datetime.now() - f_svmstart
    print("func:evaluate_svm \nExecution Time : {:2f} s".format(f_svmduration.total_seconds()))
    return {'result' : result_avg,
            'f1'     : result_f1}

In [25]:
svm_result = _eval_svm(save_file=False)
print(svm_result)
print("AVERAGE PRECISION : ", svm_result['result']['Precision'].mean())
print("AVERAGE RECALL : ", svm_result['result']['Recall'].mean())

finish fold-1
finish fold-2
finish fold-3
finish fold-4
finish fold-5
func:evaluate_svm 
Execution Time : 2.997329 s
{'result':                TP  FN  FP   TN  Precision    Recall  f1-score
labels                                                       
business       47   2   1  170   0.961946  0.951524  0.956616
entertainment  36   0   0  184   0.981818  0.980960  0.981094
politics       42   1   1  176   0.958418  0.976852  0.967151
sport          52   0   0  169   0.988517  0.996429  0.992401
tech           38   1   0  181   0.979884  0.970808  0.975064, 'f1': {'f1': {'micro': 0.9862385321100917, 'macro': 0.9744652674364683, 'weighted': 0.9744905251507652}}}
AVERAGE PRECISION :  0.9741167085826031
AVERAGE RECALL :  0.9753144304296327


In [26]:
def _eval_nb(save_file=False) :
    i = 1
    t_res = []
    list_dataframe = []
    for train_index, val_index, test_index in fold :
        f_nbstart = datetime.now()
        
        
        X_train = vectorizer.transform(shuff['clean_text'][train_index]).toarray()
        X_val = vectorizer.transform(shuff['clean_text'][val_index]).toarray()
        X_test = vectorizer.transform(shuff['clean_text'][test_index]).toarray()


        y_train = shuff['label_cat'][train_index]
        y_val = shuff['label_cat'][val_index]
        y_test = shuff['label_cat'][test_index]
        
        mb_nb = MultinomialNB(alpha = 0.6)
        mb_nb.fit(X_train, y_train)
    
        pred = mb_nb.predict(X_test)
        
        result_df = _report_classification(confusion_matrix(y_test, pred))
        
        list_dataframe.append(result_df)
        # delete
        X_train, X_val,X_test = None, None, None
        y_train, y_val,y_test = None, None, None
        
        print('finish fold-{}'.format(i))
        i += 1
    # average all fold
    result_avg = _average_dataframe(list_dataframe)
    result_f1 = get_f1(result_avg)

    f_nbduration = datetime.now() - f_nbstart
    print("func:evaluate_naivebayes \nExecution Time : {:2f} s".format(f_nbduration.total_seconds()))
    return {'result' : result_avg,
            'f1'     : result_f1}

In [27]:
nb_result = _eval_nb()
print(nb_result)

print("AVERAGE PRECISION : ", nb_result['result']['Precision'].mean())
print("AVERAGE RECALL : ", nb_result['result']['Recall'].mean())

finish fold-1
finish fold-2
finish fold-3
finish fold-4
finish fold-5
func:evaluate_naivebayes 
Execution Time : 0.485442 s
{'result':                TP  FN  FP   TN  Precision    Recall  f1-score
labels                                                       
business       47   1   2  170   0.958984  0.968059  0.963367
entertainment  35   1   0  184   0.988342  0.958754  0.973202
politics       42   1   1  177   0.971013  0.970799  0.970346
sport          52   0   0  169   0.992792  0.996429  0.994594
tech           38   1   1  180   0.955652  0.968682  0.961854, 'f1': {'f1': {'micro': 0.981651376146789, 'macro': 0.9726725014236685, 'weighted': 0.9735455827966392}}}
AVERAGE PRECISION :  0.9733564675365729
AVERAGE RECALL :  0.9725447439267587
